In [1]:
from typeconf import SelectConfig, BaseConfig
from pydantic import ValidationError
import torch
from torch import nn
from typing import Tuple, Union

Access predefined Configurations for a torch optimizer

In [2]:
from typeconf.libs.torch.optim import OptimizerConfig

Register some Non-Linearities we later want to quickly switch out.
The name passed in register is later used in the configuration to specify which one we want to use.

In [13]:
class NonLinearityConfig(SelectConfig):
    pass

# Same class must be used in register as well as in inheritance
@NonLinearityConfig.register('sigmoid')
class Sigmoid(NonLinearityConfig):
    def build(self):
        return torch.sigmoid

@NonLinearityConfig.register('tanh')
class Tanh(NonLinearityConfig):
    def build(self):
        return torch.tanh

Register some models. Here we use the NonLinearityConfig. In the build function, we dynamically get the correct non-linearity and pass it to our model

In [4]:
class ModelConfig(SelectConfig):
    pass

class ConvModel(nn.Module):
    def __init__(self, activation_fn, kernel_size):
        super().__init__()
        self.activation_fn = activation_fn
        self.conv1 = nn.Conv2d(3, 3, kernel_size=kernel_size)
    def forward(self, x):
        x = self.conv1(x)
        return self.activation_fn(x)

# Using the configuration from
@ModelConfig.register('ConvNetwork')
class ConvModelConfig(ModelConfig):
    activation_fn : NonLinearityConfig
    kernel_size : Union[Tuple[int, int], int] = (3, 3)
    def build(self):
        activation_fn = self.activation_fn.build()
        return ConvModel(activation_fn, self.kernel_size)

class LinearModel(nn.Module):
    def __init__(self, activation_fn, in_features, out_features):
        super().__init__()
        self.activation_fn = activation_fn
        self.fc1 = nn.Linear(in_features, out_features)
    def forward(self, x):
        x = self.fc1(x)
        return self.activation_fn(x)

@ModelConfig.register('LinearNetwork')
class LinearModelConfig(ModelConfig):
    activation_fn : NonLinearityConfig
    in_features : int = 10
    out_features : int = 10
    def build(self):
        activation_fn = self.activation_fn.build()
        return LinearModel(activation_fn, self.in_features, self.out_features)

We define a overall experiment configuration

In [14]:
class ExperimentConfig(BaseConfig):
    model : ModelConfig
    optimizer : OptimizerConfig

A few configurations to test out

In [16]:
def run(cfg):
    cfg = ExperimentConfig(**cfg)
    print(cfg)

    model = cfg.model.build()
    print(model) 
    print(model.activation_fn)
    optimizer = cfg.optimizer.build(model.parameters())
    print(optimizer)

In [17]:
cfg1 = {
    'model': {
        'name': 'LinearNetwork',
        'activation_fn': {
            'name': 'sigmoid'
        }
    },
    'optimizer': {
        'name': 'Adagrad'
    }
}

run(cfg1)


model=LinearModelConfig(name='LinearNetwork', activation_fn=Sigmoid(name='sigmoid'), in_features=10, out_features=10) optimizer=AdagradConfig(name='Adagrad', lr=0.01, lr_decay=0, eps=1e-10, initial_accumulator_value=0.0, weight_decay=0)
LinearModel(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
)
<built-in method sigmoid of type object at 0x10ebd1c30>
Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0.0
    lr: 0.01
    lr_decay: 0
    weight_decay: 0
)


In [18]:
cfg2 = {
    'model': {
        'name': 'ConvNetwork',
        'activation_fn': {
            'name': 'tanh'
        }
    },
    'optimizer': {
        'name': 'Adagrad'
    }
}
run(cfg2)

model=ConvModelConfig(name='ConvNetwork', activation_fn=Tanh(name='tanh'), kernel_size=(3, 3)) optimizer=AdagradConfig(name='Adagrad', lr=0.01, lr_decay=0, eps=1e-10, initial_accumulator_value=0.0, weight_decay=0)
ConvModel(
  (conv1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
)
<built-in method tanh of type object at 0x10ebd1c30>
Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0.0
    lr: 0.01
    lr_decay: 0
    weight_decay: 0
)


avoid runtime error because option does not exist

In [28]:
cfg_wrong_model_name = {
    'model': {
        'name': 'ConvNetwork',
        'activation_fn': {
            'name': 'relu'
        }
    },
    'optimizer': {
        'name': 'Adadelta'
    }
}
try:
    run(cfg_wrong_model_name)
except ValueError as e:
    print('Error!')
    print(e)

Error!
1 validation error for ExperimentConfig
model -> activation_fn
  Unknown option for NonLinearityConfig: relu (type=value_error)


In [24]:
# avoid runtime error because of invalid data type
cfg_wrong_kernel_size = {
    'model': {
        'name': 'ConvNetwork',
        'kernel_size': (3, 3, 3),
        'activation_fn': {
            'name': 'tanh'
        }
    },
    'optimizer': {
        'name': 'Adagrad'
    }
}
try:
    run(cfg_wrong_kernel_size)
except ValueError as e:
    print('Error!')
    print(e)

Error!
2 validation errors for ExperimentConfig
model -> kernel_size
  wrong tuple length 3, expected 2 (type=value_error.tuple.length; actual_length=3; expected_length=2)
model -> kernel_size
  value is not a valid integer (type=type_error.integer)


Avoid default value being used because wrong naming

In [23]:
cfg_wrong_kernel_size_name = {
    'model': {
        'name': 'ConvNetwork',
        'kernel_sizze': (3, 3),
        'activation_fn': {
            'name': 'tanh'
        }
    },
    'optimizer': {
        'name': 'Adagrad'
    }
}
try:
    run(cfg_wrong_kernel_size_name)
except ValidationError as e:
    print('Error!')
    print(e)

Error!
1 validation error for ExperimentConfig
model -> kernel_sizze
  extra fields not permitted (type=value_error.extra)


Configuration can also be modified directly from CLI

In [11]:
parser = ExperimentConfig._create_parser()

In [12]:
parser.print_help()

usage: ExperimentConfig [-h] [--config_path CONFIG_PATH]
                        [--model.name {convnetwork,linearnetwork}]
                        [--optimizer.name {adadelta,adagrad}]

optional arguments:
  -h, --help            show this help message and exit
  --config_path CONFIG_PATH
  --model.name {convnetwork,linearnetwork}
  --optimizer.name {adadelta,adagrad}
